Analysis


In [1]:
from pyspark.sql import SparkSession
output_data = 'data/output'
spark = SparkSession \
        .builder \
        .appName("ND027_Capstone_Project_query") \
        .getOrCreate()


23/04/29 11:22:12 WARN Utils: Your hostname, DESKTOP-76EIVRC resolves to a loopback address: 127.0.1.1; using 172.26.128.74 instead (on interface eth0)
23/04/29 11:22:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/29 11:22:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Create Views


In [2]:
immigration_fact_tbl = spark.read.parquet(f'{output_data}/immigration_fact')
users_dimensions_tbl = spark.read.parquet(f'{output_data}/users_dimension')
countries_dimension_tbl = spark.read.parquet(f'{output_data}/countries_dimension')
cities_demographics_dimension_tbl = spark.read.parquet(f'{output_data}/cities_demographics_dimension')

immigration_fact_view = immigration_fact_tbl.createOrReplaceTempView('immigration_fact_view')
users_dimensions_view = users_dimensions_tbl.createOrReplaceTempView('users_dimensions_view')
countries_dimension_view = countries_dimension_tbl.createOrReplaceTempView('countries_dimension_view')
cities_demographics_dimension_view = cities_demographics_dimension_tbl.createOrReplaceTempView('cities_demographics_dimension_view')

Analyze the data

In [20]:
spark.sql("""
    SELECT count(*) FROM immigration_fact_view where cicid is null
""").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [12]:
spark.sql("""
    SELECT * FROM countries_dimension_view
""").show(100)

+------------+--------------------+
|country_code|                name|
+------------+--------------------+
|         527|TURKS AND CAICOS ...|
|         420|              TUVALU|
|         352|              UGANDA|
|         162|             UKRAINE|
|         296|UNITED ARAB EMIRATES|
|         135|      UNITED KINGDOM|
|         695|             URUGUAY|
|         163|          UZBEKISTAN|
|         410|             VANUATU|
|         696|           VENEZUELA|
|         266|             VIETNAM|
|         469|WALLIS AND FUTUNA...|
|         757|WEST INDIES (FRENCH)|
|         333|      WESTERN SAHARA|
|         465|       WESTERN SAMOA|
|         216|               YEMEN|
|         139|          YUGOSLAVIA|
|         301|               ZAIRE|
|         344|              ZAMBIA|
|         315|            ZIMBABWE|
|         403|INVALID: AMERICAN...|
|         712| INVALID: ANTARCTICA|
|         700|INVALID: BORN ON ...|
|         719|INVALID: BOUVET I...|
|         574|     INVALID: 

In [4]:
spark.sql("""
    SELECT *
    FROM
        (SELECT DISTINCT
            iv.i94cit, 
            COUNT(iv.cicid) AS count_immigrants
        FROM immigration_fact_view iv
        GROUP BY iv.i94cit
        ) AS immigrants_by_country
    JOIN countries_dimension_view cv
        ON immigrants_by_country.i94cit != cv.country_code
    ORDER BY immigrants_by_country.count_immigrants DESC
""").show()

+------+----------------+------------+--------------------+
|i94cit|count_immigrants|country_code|                name|
+------+----------------+------------+--------------------+
| 135.0|           33237|         527|TURKS AND CAICOS ...|
| 135.0|           33237|         712| INVALID: ANTARCTICA|
| 135.0|           33237|         420|              TUVALU|
| 135.0|           33237|         352|              UGANDA|
| 135.0|           33237|         162|             UKRAINE|
| 135.0|           33237|         296|UNITED ARAB EMIRATES|
| 135.0|           33237|         695|             URUGUAY|
| 135.0|           33237|         163|          UZBEKISTAN|
| 135.0|           33237|         410|             VANUATU|
| 135.0|           33237|         696|           VENEZUELA|
| 135.0|           33237|         266|             VIETNAM|
| 135.0|           33237|         469|WALLIS AND FUTUNA...|
| 135.0|           33237|         757|WEST INDIES (FRENCH)|
| 135.0|           33237|         333|  

In [14]:
spark.sql("""
    SELECT i94port, count_immigrants, City, state_code, total_population, foreign_born
    FROM
        (SELECT DISTINCT
            iv.i94port, 
            COUNT(iv.cicid) AS count_immigrants
        FROM immigration_fact_view iv
        GROUP BY iv.i94port
        ) AS immigrants_by_city
    JOIN cities_demographics_dimension_view cv
        ON immigrants_by_city.i94port = cv.city_code
    ORDER BY count_immigrants DESC
""").show()

+-------+----------------+---------------+----------+----------------+------------+
|i94port|count_immigrants|           City|state_code|total_population|foreign_born|
+-------+----------------+---------------+----------+----------------+------------+
|    NYC|           32779|       New York|        NY|        42752025|    16062500|
|    MIA|           22660|          Miami|        FL|         2204945|     1303945|
|    LOS|           20983|    Los Angeles|        CA|        19859480|     7427125|
|    ORL|           12332|        Orlando|        FL|         1354585|      252790|
|    CHI|           11823|        Chicago|        IL|        13602780|     2867315|
|    SFR|           11056|  San Francisco|        CA|         4324080|     1485995|
|    FTL|            8087|Fort Lauderdale|        FL|          892935|      237910|
|    HOU|            7305|        Houston|        TX|        11493140|     3481050|
|    LVG|            7216|      Las Vegas|        NV|         3118845|      

In [13]:
spark.sql("""
    SELECT iv.* FROM immigration_fact_view iv
    JOIN countries_dimension_view cv
    ON iv != cv.country_code
""").show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `iv` cannot be resolved. Did you mean one of the following? [`iv`.`cicid`, `iv`.`i94yr`, `cv`.`name`, `iv`.`i94cit`, `iv`.`i94mon`].; line 4 pos 7;
'Project [iv.*]
+- 'Join Inner, NOT ('iv = country_code#40)
   :- SubqueryAlias iv
   :  +- SubqueryAlias immigration_fact_view
   :     +- View (`immigration_fact_view`, [cicid#0,i94cit#1,i94res#2,i94port#3,arrdate#4,i94mode#5,i94addr#6,depdate#7,i94visa#8,visatype#9,dtaddto#10,arrival_date_ts#11,departure_date_ts#12,i94yr#13,i94mon#14])
   :        +- Relation [cicid#0,i94cit#1,i94res#2,i94port#3,arrdate#4,i94mode#5,i94addr#6,depdate#7,i94visa#8,visatype#9,dtaddto#10,arrival_date_ts#11,departure_date_ts#12,i94yr#13,i94mon#14] parquet
   +- SubqueryAlias cv
      +- SubqueryAlias countries_dimension_view
         +- View (`countries_dimension_view`, [country_code#40,name#41])
            +- Relation [country_code#40,name#41] parquet
